In [1]:
import findspark
findspark.init()

import pyspark
import random
from pathlib import Path
from pyspark.sql import SparkSession

In [2]:
spark=SparkSession.builder.appName('lreg2').getOrCreate()

In [3]:
from pyspark.ml.regression import LinearRegression
path = Path('D:/Python environments/Pyspark_env/environments')

In [31]:
# D:\Python environments\Pyspark_env\environments\Data\Python-and-Spark-for-Big-Data-master\Spark_for_Machine_Learning\Linear_Regression

In [4]:
data=spark.read.csv(f'{path}/data/Python-and-Spark-for-Big-Data-master/Spark_for_Machine_Learning/Linear_Regression/Ecommerce_Customers.csv',
                       inferSchema=True,header=True)

In [5]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [9]:
for item in data.head(1)[0]:
    print(item,'\n')

mstephenson@fernandez.com 

835 Frank TunnelWrightmouth, MI 82180-9605 

Violet 

34.49726772511229 

12.65565114916675 

39.57766801952616 

4.0826206329529615 

587.9510539684005 



In [11]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [12]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [13]:
assembler=VectorAssembler(inputCols=['Avg Session Length','Time on App','Time on Website','Length of Membership'],
                         outputCol='features')

In [14]:
output=assembler.transform(data)
output.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)
 |-- features: vector (nullable = true)



In [15]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [16]:
final_data=output.select(['features','Yearly Amount Spent'])
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [17]:
train_data,test_data=final_data.randomSplit([0.7,0.3])

In [18]:
print(train_data.describe().show())
print(test_data.describe().show())

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                350|
|   mean| 498.17126825107994|
| stddev|  79.72268138531581|
|    min|   266.086340948469|
|    max|  765.5184619388373|
+-------+-------------------+

None
+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                150|
|   mean|  501.9805016094491|
| stddev|  78.55496540082046|
|    min| 256.67058229005585|
|    max|  725.5848140556806|
+-------+-------------------+

None


In [19]:
lr=LinearRegression(featuresCol='features',labelCol='Yearly Amount Spent',predictionCol='predictions')

In [20]:
lr_model=lr.fit(train_data)

In [21]:
test_results=lr_model.evaluate(test_data)

In [24]:
test_results.residuals.show()

+-------------------+
|          residuals|
+-------------------+
| 11.043431375818614|
| -3.676571176511459|
| -6.177816576507951|
| -3.741107380530593|
| -8.281487948091467|
| 18.535883951821233|
|-0.8461384680484798|
| -0.844861651755707|
|  1.198322345464078|
|-12.009562581570435|
| -4.146916640722566|
| -16.08902751812394|
|  7.353977554467008|
|  8.659082871122393|
| -6.240545712948176|
|  4.217828218107968|
|   5.61229253623776|
|  6.081843479464737|
|   16.8304408554813|
|    9.7471469639583|
+-------------------+
only showing top 20 rows



In [25]:
test_results.r2

0.9827133163769927

In [26]:
test_results.rootMeanSquaredError

10.293829916975826

In [27]:
unlabeled_data=test_data.select('features')
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.7377203726281...|
|[30.8364326747734...|
|[31.0613251567161...|
|[31.2681042107507...|
|[31.4474464941278...|
|[31.6005122003032...|
|[31.6610498227460...|
|[31.7216523605090...|
|[31.7366356860502...|
|[31.8093003166791...|
|[31.8124825597242...|
|[31.8164283341993...|
|[31.8209982016720...|
|[31.8512531286083...|
|[31.8745516945853...|
|[32.0123007682454...|
|[32.0305497162129...|
|[32.0478009788678...|
|[32.0478146331398...|
|[32.1151190660142...|
+--------------------+
only showing top 20 rows



In [30]:
pred=lr_model.transform(unlabeled_data)
pred.show()

+--------------------+------------------+
|            features|       predictions|
+--------------------+------------------+
|[30.7377203726281...|450.73731082041127|
|[30.8364326747734...|471.17847160350107|
|[31.0613251567161...|493.73327463440955|
|[31.2681042107507...| 427.2116405543545|
|[31.4474464941278...| 426.8842300433155|
|[31.6005122003032...| 460.6369675392757|
|[31.6610498227460...| 417.2044920479493|
|[31.7216523605090...|348.62178828362835|
|[31.7366356860502...|495.73512391006784|
|[31.8093003166791...| 548.7814619444116|
|[31.8124825597242...| 396.9572616245198|
|[31.8164283341993...| 517.2115190217803|
|[31.8209982016720...|417.32130345874634|
|[31.8512531286083...|  464.333163795676|
|[31.8745516945853...| 398.5257899592157|
|[32.0123007682454...|488.72722484785027|
|[32.0305497162129...|  588.662190882374|
|[32.0478009788678...| 507.3687277066317|
|[32.0478146331398...| 480.5591169033621|
|[32.1151190660142...| 340.3110531998868|
+--------------------+------------